## Import requirements & Parameters setting

In [1]:
import os
import random
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, LSTM, TimeDistributed, Input, ReLU, Bidirectional
from sklearn.metrics import mean_absolute_error, mean_squared_error

batch_size = 32
epochs = 1000

# set model name & model directory
model_name = 'baseline'

# get csv files
csvs = []
folders = glob.glob('data/Trajectory-11/data*')
for folder in folders:
    trajectories = glob.glob(folder+'/*')
    for trajectory in trajectories:
        csvs.extend(glob.glob(trajectory+'/*.csv'))

# shuffle
random.Random(4).shuffle(csvs)

# split train / valid / test
length = len(csvs)
train_list = csvs[:int(length*0.8)]
valid_list = csvs[int(length*0.8):int(length*0.9)]
test_list = csvs[int(length*0.9):length]

print('train: {}'.format(len(train_list)))
print('valid: {}'.format(len(valid_list)))
print('test: {}'.format(len(test_list)))

train: 288
valid: 36
test: 36


## Dynamic allocation for GPU (optional)

In [2]:
import keras.backend.tensorflow_backend as KTF

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   # Dynamic allocation
sess = tf.Session(config=config)
KTF.set_session(sess)

Using TensorFlow backend.


## Train / Validation / Test generator definition

In [3]:
def train_generator():
    while(True):
        for i, csv in enumerate(train_list):
            df = pd.read_csv(csv)
            data = []
            label = []
            for idx in df.index:
                data.append(df.loc[idx, ['D1', 'D2', 'D3']].tolist())
                label.append(df.loc[idx, ['gX', 'gY']].to_numpy() / [428, 300])
            
            x_train = np.expand_dims(data, axis=0)
            y_train = np.expand_dims(np.array(label), axis=0)
            y_train = y_train.astype(np.float32)            
            
            yield x_train, y_train, [None]

In [4]:
def valid_generator():
    while(True):
        for i, csv in enumerate(valid_list):
            df = pd.read_csv(csv)
            data = []
            label = []
            for idx in df.index:
                data.append(df.loc[idx, ['D1', 'D2', 'D3']].tolist())
                label.append(df.loc[idx, ['gX', 'gY']].to_numpy() / [428, 300])
            
            x_valid = np.expand_dims(data, axis=0)
            y_valid = np.expand_dims(np.array(label), axis=0)
            y_valid = y_valid.astype(np.float32)
            
            yield x_valid, y_valid, [None]

In [5]:
def test_generator():
    for i, csv in enumerate(test_list):
        df = pd.read_csv(csv)
        df = df.iloc[::-1]
        data = []
        label = []
        triangulation = []
        for idx in df.index:
            data.append(df.loc[idx, ['D1', 'D2', 'D3']].tolist())
            label.append(df.loc[idx, ['gX', 'gY']].to_numpy())
            triangulation.append(df.loc[idx, ['rX', 'rY']].to_numpy())
        
        x_test = np.expand_dims(data, axis=0)
        y_test = np.expand_dims(np.array(label), axis=0)
        y_test = y_test.astype(np.float32)
        triangulation = np.array(triangulation)
        
        yield x_test, y_test, triangulation

## Build model

In [6]:
# Build customized loss
from keras import backend as K

def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [7]:
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(11, 3)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(256)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(2)))
model.add(ReLU())
model.compile(loss=rmse, optimizer="adam", metrics=[rmse])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 11, 256)           135168    
_________________________________________________________________
dropout (Dropout)            (None, 11, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 11, 512)           1050624   
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 512)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 11, 256)           131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 11, 256)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 11, 2)             514       
__________

In [8]:
# plot model
from tensorflow.keras.utils import plot_model
plot_file = os.path.join('models', model_name + '.png')
plot_model(model, show_shapes=True, show_layer_names=True, to_file=plot_file)

In [9]:
# save training logs
log_dir = os.path.join(os.getcwd(), 'logs')
model_cbk = keras.callbacks.TensorBoard(log_dir="logs/{}".format(model_name), histogram_freq=1)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')

# save best model weights
model_mckp = keras.callbacks.ModelCheckpoint(os.path.join('models', model_name + '.h5'), 
                                             monitor='val_loss', 
                                             save_best_only=True, 
                                             mode='min')

In [10]:
history = model.fit(train_generator(), 
                    steps_per_epoch=batch_size, 
                    epochs=epochs, 
                    validation_data=valid_generator(),
                    validation_steps=len(valid_list),
                    callbacks=[model_mckp, model_cbk])

Epoch 1/1000
32/32 [==============================] - 8s 245ms/step - loss: 0.4587 - rmse: 0.4587 - val_loss: 0.4110 - val_rmse: 0.4110
Epoch 2/1000
32/32 [==============================] - 5s 150ms/step - loss: 0.4771 - rmse: 0.4771 - val_loss: 0.2932 - val_rmse: 0.2932
Epoch 3/1000
32/32 [==============================] - 5s 143ms/step - loss: 0.3819 - rmse: 0.3819 - val_loss: 0.4118 - val_rmse: 0.4118
Epoch 4/1000
32/32 [==============================] - 5s 141ms/step - loss: 0.3912 - rmse: 0.3912 - val_loss: 0.3533 - val_rmse: 0.3533
Epoch 5/1000
32/32 [==============================] - 5s 145ms/step - loss: 0.3625 - rmse: 0.3625 - val_loss: 0.3447 - val_rmse: 0.3447
Epoch 6/1000
32/32 [==============================] - 5s 143ms/step - loss: 0.3124 - rmse: 0.3124 - val_loss: 0.3213 - val_rmse: 0.3213
Epoch 7/1000
32/32 [==============================] - 4s 140ms/step - loss: 0.3716 - rmse: 0.3716 - val_loss: 0.3247 - val_rmse: 0.3247
Epoch 8/1000
32/32 [============================

## Model evaluate

In [11]:
# Predict
predict = model.predict_generator(test_generator(), verbose=1, steps=len(test_list))

# Present as dataframe
df = pd.DataFrame()
for idx, (_, y_test, triangulation) in enumerate(test_generator()):
    for timestep, (gt, tri, pred) in enumerate(zip(y_test[0], triangulation, predict[idx])):
        df.loc[idx, 'gX{}'.format(timestep)] = gt[0]
        df.loc[idx, 'gY{}'.format(timestep)] = gt[1]
        df.loc[idx, 'rX{}'.format(timestep)] = tri[0]
        df.loc[idx, 'rY{}'.format(timestep)] = tri[1]
        df.loc[idx, 'pX{}'.format(timestep)] = pred[0]
        df.loc[idx, 'pY{}'.format(timestep)] = pred[1]
        loss = mean_squared_error(df.loc[idx, ['gX{}'.format(timestep), 'gY{}'.format(timestep)]].tolist(),
                                  df.loc[idx, ['pX{}'.format(timestep), 'pY{}'.format(timestep)]].tolist())
        df.loc[idx, 'Loss{}'.format(timestep)] = loss**0.5
pd.set_option("display.precision", 2)
df.T

36/36 [==============================] - 2s 58ms/step


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
gX0,202.00,242.00,54.00,113.00,208.00,129.00,129.00,96.00,2.61e+02,2.47e+02,...,72.00,274.00,236.00,41.00,193.00,78.00,94.00,69.00,239.00,248.00
gY0,420.00,400.00,222.00,369.00,216.00,163.00,333.00,392.00,6.40e+01,6.50e+01,...,193.00,310.00,326.00,347.00,387.00,30.00,351.00,365.00,356.00,361.00
rX0,227.43,205.87,273.55,150.82,219.84,144.04,110.65,41.15,2.00e+02,2.02e+02,...,74.34,287.69,234.38,16.29,127.23,86.33,97.52,79.82,199.19,201.88
rY0,421.25,421.26,-4.41,362.86,412.01,128.42,345.72,390.07,9.07e+01,9.44e+01,...,235.21,293.53,350.42,353.48,412.43,-9.29,367.08,345.21,406.28,410.97
pX0,0.51,0.51,0.17,0.45,0.51,0.18,0.29,0.28,4.77e-01,4.79e-01,...,0.16,0.50,0.56,0.21,0.43,0.13,0.27,0.27,0.53,0.54
pY0,1.19,1.16,0.60,1.18,1.18,0.57,1.08,1.19,0.00e+00,0.00e+00,...,0.65,0.99,1.11,1.15,1.19,0.14,1.07,1.17,1.18,1.18
Loss0,328.64,329.69,161.11,271.99,211.19,146.59,251.73,284.51,1.90e+02,1.80e+02,...,145.19,291.80,283.71,246.25,304.90,58.98,256.16,261.82,302.30,308.79
gX1,216.00,256.00,58.00,123.00,214.00,121.00,131.00,106.00,2.41e+02,2.27e+02,...,78.00,256.00,238.00,77.00,171.00,78.00,90.00,61.00,238.00,234.00
gY1,416.00,370.00,242.00,361.00,234.00,177.00,323.00,392.00,7.60e+01,7.90e+01,...,213.00,290.00,312.00,354.00,395.00,48.00,343.00,383.00,372.00,377.00
rX1,215.80,227.77,276.98,168.64,218.96,140.25,112.76,48.47,1.89e+02,1.82e+02,...,73.75,285.12,236.59,23.01,102.32,86.75,93.03,75.15,182.81,170.92


In [12]:
# Caculate mean loss
last_idx = len(df)
for timestep in range(11):
    df.loc[last_idx, 'Loss{}'.format(timestep)] = df['Loss{}'.format(timestep)].mean()

# Save result to csv
df.T.to_csv(os.path.join('predictions', model_name + '.csv'))
df.T.tail()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
rX10,279.18,277.69,285.29,256.14,287.24,104.24,157.09,186.83,57.99,48.97,...,225.26,262.63,201.35,57.20,111.56,92.53,182.52,81.52,76.95,NaN
rY10,331.04,264.38,130.15,339.72,302.20,273.07,236.79,410.20,35.52,39.76,...,217.64,239.57,400.46,368.59,132.91,231.03,422.60,339.83,348.57,NaN
pX10,0.53,0.52,0.26,0.57,0.52,0.26,0.31,0.52,0.12,0.12,...,0.49,0.51,0.52,0.24,0.24,0.19,0.51,0.29,0.28,NaN
pY10,0.96,0.93,1.02,1.12,0.93,0.98,0.77,1.17,0.20,0.20,...,0.96,0.95,1.17,1.20,0.42,0.62,1.17,1.18,1.18,NaN
Loss10,266.27,234.09,279.69,306.89,333.24,247.25,175.81,318.86,65.12,66.96,...,202.98,218.64,302.54,273.37,179.28,141.86,313.91,264.99,279.47,229.6
